<a href="https://colab.research.google.com/github/IndraMadika/ML_Study/blob/main/Membangun_model_yang_bagus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf
from tensorflow.keras import layers

# Membangun arsitektur model CNN
input_shape = (224, 224, 3)

# Membuat input layer
inputs = tf.keras.Input(shape=input_shape)

# Membuat model pretrained menggunakan weight dari ImageNet
pretrained_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
pretrained_model.summary()
print("================================================")



Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_8[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

Normalisasi batch (batch normalization) memiliki beberapa kegunaan dalam model di atas:

Stabilisasi pelatihan: Normalisasi batch membantu dalam menjaga stabilitas pelatihan model. Dalam setiap batch data, normalisasi batch memusatkan dan menormalkan fitur input ke dalam rentang yang lebih kecil. Hal ini membantu dalam mencegah perubahan drastis pada distribusi input yang dapat mempengaruhi pelatihan model.

Percepatan pelatihan: Dengan menggunakan normalisasi batch, pelatihan model dapat berjalan lebih cepat. Normalisasi batch secara internal menghitung statistik batch seperti mean dan variansi untuk setiap fitur dan melakukan normalisasi berdasarkan statistik tersebut. Ini membantu dalam mempercepat proses pelatihan dan mengurangi ketergantungan pada bobot dan bias yang harus diperbarui secara iteratif.

Penyederhanaan inisialisasi: Normalisasi batch dapat membantu dalam penyederhanaan proses inisialisasi model. Dalam beberapa kasus, normalisasi batch dapat mengurangi kebutuhan untuk melakukan penyesuaian inisialisasi yang kompleks dan mengurangi kerentanan terhadap inisialisasi yang buruk.

Regularisasi: Normalisasi batch memiliki efek regularisasi yang dapat membantu dalam mencegah overfitting. Ini terjadi karena normalisasi batch memperkenalkan sedikit kebisingan pada setiap batch data, yang dapat membantu dalam menggeneralisasi model dengan lebih baik.

Dalam model di atas, normalisasi batch diterapkan setelah lapisan konvolusi dan sebelum aktivasi ReLU. Hal ini membantu dalam meningkatkan stabilitas dan kecepatan pelatihan model, serta dapat membantu dalam mencegah overfitting

In [6]:
# Menggunakan output layer dari model pretrained sebagai input untuk model kita
x = pretrained_model(inputs)

# Konvolusi pertama
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)

# Blok residual pertama
block_1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
block_1 = layers.BatchNormalization()(block_1)
block_1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(block_1)
block_1 = layers.BatchNormalization()(block_1)
x = layers.Add()([x, block_1])

# Konvolusi kedua
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)

# Blok residual kedua
block_2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
block_2 = layers.BatchNormalization()(block_2)
block_2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(block_2)
block_2 = layers.BatchNormalization()(block_2)
x = layers.Add()([x, block_2])

# Max pooling
x = layers.MaxPooling2D((2, 2))(x)

# Flatten layer
x = layers.Flatten()(x)

# Fully connected layer
x = layers.Dense(256, activation='relu')(x)
x = layers.BatchNormalization()(x)
outputs = layers.Dense(10, activation='softmax')(x)

# Membuat model gabungan
model = tf.keras.Model(inputs, outputs)

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Menampilkan ringkasan model
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 resnet50 (Functional)          (None, 7, 7, 2048)   23587712    ['input_7[0][0]']                
                                                                                                  
 conv2d_21 (Conv2D)             (None, 7, 7, 64)     1179712     ['resnet50[0][0]']               
                                                                                                  
 batch_normalization_24 (BatchN  (None, 7, 7, 64)    256         ['conv2d_21[0][0]']        

Kode x = layers.Add()([x, block_1]) melakukan operasi penggabungan (addition) antara x (output sebelumnya) dan block_1 (output dari blok residual pertama).

Penggabungan ini merupakan langkah kunci dalam arsitektur ResNet yang dikenal sebagai shortcut connection atau skip connection. Tujuan dari penggabungan ini adalah untuk memungkinkan aliran informasi yang lancar melalui jaringan, memfasilitasi pembelajaran yang lebih baik, dan mengatasi masalah degradasi kinerja saat meningkatkan kedalaman model.

Penggabungan dilakukan dengan menjumlahkan output sebelumnya (x) dengan output dari blok residual (block_1). Melalui penggabungan ini, model dapat mempelajari perbedaan antara input dan output pada setiap lapisan, sehingga memungkinkan model untuk mempelajari representasi yang lebih baik dan mengoptimalkan fitur yang relevan.

Penggabungan ini memiliki beberapa keuntungan:

Mencegah degradasi kinerja: Dalam beberapa kasus, penambahan lapisan yang lebih dalam pada model dapat menyebabkan penurunan kinerja. Penggabungan melalui shortcut connection memungkinkan informasi asli untuk dilewatkan langsung ke lapisan berikutnya, sehingga membantu mengatasi masalah degradasi kinerja tersebut.

Memudahkan pelatihan: Shortcut connection mempermudah pelatihan model dengan mempercepat aliran gradien dan mengurangi kesulitan dalam propagasi gradien selama pelatihan. Hal ini memungkinkan model untuk lebih efisien belajar dari data pelatihan.

Meningkatkan akurasi: Dengan memungkinkan aliran informasi yang lancar melalui jaringan, penggabungan melalui shortcut connection dapat membantu model mempelajari fitur yang lebih baik dan menghasilkan prediksi yang lebih akurat.

Dengan demikian, penggabungan melalui shortcut connection dalam model ResNet merupakan komponen penting yang membantu dalam meningkatkan kemampuan model untuk mempelajari representasi yang lebih baik, meningkatkan stabilitas pelatihan, dan dapat berkontribusi pada peningkatan akurasi model.